In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
pd.options.display.max_columns = None

In [4]:
CRS = "EPSG:4326"

In [5]:
shape_gdf = gpd.read_file("data/BRA_adm2.shp").rename(
    columns={"NAME_2": "muni_name", "ID_2": "muni_id"}
)[["muni_id", "muni_name", "geometry"]]

In [6]:
aid_df = (
    pd.read_csv("data/aid.csv").rename(
        columns={"ids": "aid_id", "latitude": "aid_lat", "longitude": "aid_lon"}
    )
    # take only the program level of aid_id
    .assign(aid_id=lambda df_: df_["aid_id"].str.split("_", expand=True)[0])
)

poll_df = pd.read_csv("data/polling_stations.csv").rename(
    columns={"local_id": "poll_id", "lat": "poll_lat", "lon": "poll_lon"}
)

assert poll_df["poll_id"].is_unique

In [7]:
aid_gdf = gpd.GeoDataFrame(
    aid_df, geometry=gpd.points_from_xy(aid_df["aid_lon"], aid_df["aid_lat"], crs=CRS)
)

poll_gdf = gpd.GeoDataFrame(
    poll_df,
    geometry=gpd.points_from_xy(poll_df["poll_lon"], poll_df["poll_lat"], crs=CRS),
)

In [8]:
# There are some polling and aid locations
# that are very slightly outside of the shape.

# The strategy is to find the nearest municipality.
# Since the distance is so slight,
# I don't bother to reproject onto a geometric CRS.

# I use an inner join and then check the number of rows,
# in order to confirm there are no locations outside
# of the shapes by more than the distance.

# roughly 10 km
max_distance = 0.05

aid_muni_gdf = gpd.sjoin_nearest(
    aid_gdf, shape_gdf, how="inner", max_distance=max_distance
)[["aid_id", "aid_lat", "aid_lon", "geometry", "muni_id", "muni_name"]]

poll_muni_gdf = gpd.sjoin_nearest(
    poll_gdf, shape_gdf, how="inner", max_distance=max_distance
)[["poll_id", "poll_lat", "poll_lon", "geometry", "muni_id", "muni_name"]]

assert len(aid_muni_gdf) == len(aid_df)
assert len(poll_muni_gdf) == len(poll_df)

/Users/edwardkrueger/.pyenv/versions/3.10.2/lib/python3.10/site-packages/geopandas/array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/Users/edwardkrueger/.pyenv/versions/3.10.2/lib/python3.10/site-packages/geopandas/array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [9]:
# Naive method causes the kernel to crash.
# Passing `dropna=False` to `.pivot_table` causes the kernel to crash.

# The strategy is to calculate the design matrix in two parts:
# * where polling locations that have aid locations in their municipality
# * where they don't
# I then concatenate the results and verify that no rows are missing.

inner_df = pd.merge(poll_muni_gdf, aid_muni_gdf, on="muni_id", how="inner")

inner_design_df = (
    inner_df.loc[:, ["poll_id", "poll_lat", "poll_lon", "aid_id"]]
    .assign(aid_id_copy=lambda df_: df_["aid_id"])
    .pivot_table(
        index=["poll_id", "poll_lat", "poll_lon"],
        columns="aid_id",
        values="aid_id_copy",
        aggfunc=len,
    )
)

left_only_design_df = (
    pd.merge(poll_muni_gdf, aid_muni_gdf, on="muni_id", how="left")
    .loc[lambda df_: df_["aid_id"].isna(), ["poll_id", "poll_lat", "poll_lon"]]
    .set_index(["poll_id", "poll_lat", "poll_lon"])
)

poll_design_df = pd.concat([inner_design_df, left_only_design_df])

assert len(poll_design_df) == len(poll_df)

In [10]:
poll_design_df

,,,p104995,p108654,p111512,p114204,p116170,p118410,p120391,p125630,p125829
poll_id,poll_lat,poll_lon,,,,,,,,,
1858,-7.116051,-34.885323,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1859,-7.124756,-34.882206,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1860,-7.064616,-34.841911,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1861,-7.126427,-34.899418,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1912,-19.915630,-43.917461,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
588328,-9.851097,-54.632366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610388,-9.948129,-54.953014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590293,-9.952096,-54.909950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
